In [1]:
# 회귀
# 출력의 개수를 1개로
# 손실함수를 MSE나 기타 등등..
# 데이터셋과 데이터로드를 커스텀하게 정의해서 사용하면 나머지는 동인한 패턴으로 학습평가를 진행하게 됨

In [2]:
# %pip install torch

In [5]:
# from torch.optim import Adam
# from torchvision import datasets, transforms
from torch.utils.data import Dataset , DataLoader
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

class BostonDataSet(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(X,dtype = torch.float32)
        self.y = torch.tensor(y,dtype = torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        return self.X[index] ,  self.y[index]
    
X_dataset = BostonDataSet(data , target) # 데이터를 X , y 를 한쌍으로 묶어버림 zip같은 느낌
X_train_loader = DataLoader(X_dataset , batch_size=32 , shuffle=True)

# 회귀 모델 정의
class BostonRegression(nn.Module):
    def __init__(self ,input_dim ):
        super(BostonRegression , self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim , 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32 ,1)
        )
    def forward(self , x):
        return self.model(x)
    

from torch.optim import Adam
from tqdm import tqdm
model = BostonRegression(data.shape[1])
criterion = nn.MSELoss()
optim = Adam(model.parameters() , lr = 1e-3)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

lr = 1e-3
optim = Adam(model.parameters(), lr=lr)
epochs = 20
# 학습루프
for epoch in range(epochs):
    tqdm_obj = tqdm(X_train_loader,desc=f'epoch : {epoch+1}/{epochs}')
    for data, label in tqdm_obj:
        optim.zero_grad()
        preds = model(data.to(device))
        loss = criterion(preds, label.to(device))
        loss.backward()
        optim.step()

        tqdm_obj.set_postfix(loss=loss.item())
        

torch.save(model.state_dict(), 'BostonRegression.pth') 

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
C:\Users\playdata\AppData\Local\Temp\ipykernel_4512\82714452.py:10: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
epoch : 1/20:   0%|          | 0/16 [00:00<?, ?it/s]d:\pandas_main\.venv\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
epoch : 1/20:   0%|          | 0/16 [00:00<?, ?it/s, loss=141] d:\pandas_main\.venv\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([26])) that is different to the input size (torch.Size([26, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure